In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight

from tensorflow.keras import Sequential
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

from cnn_utils import get_labels_and_sub_images, get_labels, display_result_metrics

In [ ]:
# Load Model
resnet = ResNet50V2(
    include_top=False, 
    weights='imagenet', 
    input_shape=(150, 150, 3), 
    pooling='max', 
    classes=2
)

# Freeze layers
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
# Create ResNet Model
resnet_model = Sequential()
resnet_model.add(resnet)
resnet_model.add(Flatten())

# Add additional layers
additional_layers_nodes = [1024, 512, 256, 128, 64, 32, 16]
for node_amount in additional_layers_nodes:
    resnet_model.add(
        Dense(node_amount, activation="relu")
    )

# Add final node 
resnet_model.add(
    Dense(1, activation="sigmoid")
)


In [ ]:
# Load Data
data_path = "../Data"

train_img_data = np.load(f"{data_path}/train_images.npy", allow_pickle=True)
train_bb_data = np.load(f"{data_path}/train_bb_data.npy", allow_pickle=True)

val_img_data  = np.load(f"{data_path}/val_images.npy", allow_pickle=True)
val_bb_data = np.load(f"{data_path}/val_bb_data.npy", allow_pickle=True)

In [ ]:
seal_threshold = .3

# Separate the images and image data between seal and no seal
label_1_img, label_1, label_0_img, label_0 = get_labels_and_sub_images(train_img_data, train_bb_data, threshold=seal_threshold)

# Get the labels for the validation data
val_label = get_labels(val_bb_data, seal_threshold)

In [ ]:
transfomed_images = []
transformed_labels = []

#Add mirror images and horizontal flip
for sub_image in label_1_img:
    
    # Apply tranformations
    mirrored_image = np.fliplr(sub_image)
    horizontal_flipped_image = np.flipud(sub_image)

    # Add new data
    transfomed_images.append(mirrored_image)
    transfomed_images.append(horizontal_flipped_image)

    transformed_labels.append(1)
    transformed_labels.append(1)

# Combine images
label_1_img += transfomed_images
label_1 += transformed_labels

In [ ]:
total_labels = np.array(label_1 + label_0)
total_images = np.array(label_1_img + label_0_img)

In [ ]:
unique_classes = np.unique(total_labels)
weights = class_weight.compute_class_weight(
    "balanced", 
    classes=unique_classes, 
    y=total_labels
)
class_weights = {0:weights[0], 1:weights[1]}

In [ ]:
# Compile model
resnet_model.compile(
    optimizer=Adam(learning_rate=3e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train Model
resnet_model.fit(
    total_images, 
    total_labels, 
    verbose= 1, 
    epochs = 10, 
    batch_size=150, 
    shuffle=True, 
    class_weight=class_weights
)

In [ ]:
# Get Predictions for validation set
validation_raw_predictions = (
    resnet_model
    .predict(val_img_data)
    .flatten()
)

# Convert probalities of a sub-image containing a seal into labels
validation_label_predictions = [1 if pred > .5 else 0 for pred in validation_raw_predictions]

# Display Evaluation Metrics
display_result_metrics(val_label, validation_label_predictions)

In [ ]:
model_name = "Test"
resnet_model.save(f"../Models/Tensorflow/{model_name}")